In [1]:
# Reassembling the full script with the updated API call function

# Function to make API call to GPT-3.5 Turbo with error handling and retry logic
def gpt_pairwise_comparison_with_retry(prompt, api_key):
    # Define the API endpoint for GPT-3.5 Turbo
    api_url = "https://api.openai.com/v1/engines/text-davinci-003/completions"

    # Set up the headers with your OpenAI API key
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }

    # Define the data payload for the API request
    data = {
        "prompt": prompt,
        "max_tokens": 50  # Adjust as needed based on your prompt
    }

    attempts = 0
    max_attempts = 3  # Define the maximum number of attempts

    while attempts < max_attempts:
        try:
            response = requests.post(api_url, headers=headers, json=data, timeout=60)  # 60-second timeout
            if response.status_code == 200:
                return response.json()['choices'][0]['text'].strip()
            else:
                print(f"Error in API call: {response.status_code}. Retrying...")
                attempts += 1
                time.sleep(5)  # Wait for 5 seconds before retrying
        except requests.exceptions.Timeout:
            print("Request timed out. Retrying...")
            attempts += 1
        except requests.exceptions.RequestException as e:
            print(f"An error occurred: {e}. Retrying...")
            attempts += 1
            time.sleep(5)

    return None  # Return None if all attempts fail

# Function to perform pairwise comparison for disciplines
def compare_disciplines(abstract, disciplines, base_folder):
    discipline_scores = {discipline: 0 for discipline in disciplines}
    for discipline1, discipline2 in itertools.combinations(disciplines, 2):
        prompt_file = f"{discipline1}_vs_{discipline2}.txt"
        prompt_path = os.path.join(base_folder, prompt_file)
        if os.path.exists(prompt_path):
            with open(prompt_path, 'r') as file:
                prompt = file.read().format(abstract)
                selected_discipline = gpt_pairwise_comparison_with_retry(prompt)
                if selected_discipline in discipline_scores:
                    discipline_scores[selected_discipline] += 1
    return discipline_scores

# Function to perform pairwise comparison for subdisciplines
def compare_subdisciplines(abstract, discipline, subdisciplines, base_folder):
    subdiscipline_scores = {subdiscipline: 0 for subdiscipline in subdisciplines}
    discipline_folder = os.path.join(base_folder, discipline)
    for sub1, sub2 in itertools.combinations(subdisciplines, 2):
        prompt_file = f"{sub1}_vs_{sub2}.txt"
        prompt_path = os.path.join(discipline_folder, prompt_file)
        if os.path.exists(prompt_path):
            with open(prompt_path, 'r') as file:
                prompt = file.read().format(abstract)
                selected_subdiscipline = gpt_pairwise_comparison_with_retry(prompt)
                subdiscipline = ''
                for sd in subdiscipline_scores.keys():
                    if selected_subdiscipline in sd:
                        subdiscipline == sd
                        break
                if subdiscipline:    
                    subdiscipline_scores[subdiscipline] += 1
    return subdiscipline_scores

# Main function to process the test file
def process_test_file(test_file_path, base_prompt_folder, discipline_data):
    results_dict = {}

    # Read the test file
    with open(test_file_path, mode='r') as file:
        csv_reader = csv.DictReader(file)
        for row in csv_reader:
            pubid = row['PUBID']
            abstract = row['ABSTRACT']

            # Compare disciplines
            disciplines = list(discipline_data.keys())
            discipline_scores = compare_disciplines(abstract, disciplines, base_prompt_folder)

            # Compare subdisciplines for selected disciplines
            for discipline, score in discipline_scores.items():
                if score >= 4 and discipline in discipline_data:
                    subdisciplines = list(discipline_data[discipline].keys())
                    subdiscipline_scores = compare_subdisciplines(abstract, discipline, subdisciplines, base_prompt_folder)
                    discipline_scores[discipline] = {'score': score, 'subdisciplines': subdiscipline_scores}
                else:
                    discipline_scores[discipline] = {'score': score, 'subdisciplines': {}}

            # Update results
            results_dict[pubid] = discipline_scores

    return results_dict

# Assuming the test file and discipline data are available
test_file_path = 'H:/data/WoS_data/data_gpt10.csv'
base_prompt_folder = 'prompts/pairwise_comparison'

# Process the test file and generate the result dictionary
results = process_test_file(test_file_path, base_prompt_folder, discipline_data)

# Save the results to a JSON file
output_file_path = 'chatgpt_hierarchical_pairwise.json'
with open(output_file_path, 'w') as json_file:
    json.dump(results, json_file, indent=4)

output_file_path


NameError: name 'discipline_data' is not defined

In [27]:
import csv
import itertools
import requests
import json
import os
import time
from tqdm import tqdm
a = ''
b = ''
c = ''
# Define the function for GPT-3.5 Turbo API calls with retry logic
def gpt_pairwise_comparison_with_retry(prompt, accepted_answers, api_key='api key'):
    # Define the API endpoint for GPT-3.5 Turbo
    global a,b,c
    api_url = "https://api.openai.com/v1/chat/completions"

    # Set up the headers with your OpenAI API key
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }

    # Define the data payload for the API request
    data = {
        "model": "gpt-3.5-turbo",
        "messages": [{"role": "user",
                      "content": prompt}],
        "max_tokens": 20  # Adjust as needed
    }

    attempts = 0
    max_attempts = 10  # Maximum number of attempts
    while attempts < max_attempts:
        try:
            response = requests.post(api_url, headers=headers, json=data, timeout=60)  # 60-second timeout
            if response.status_code == 200:
                response = response.json()['choices'][0]['message']['content'].strip(' .,\'').lower()
                if response in accepted_answers[0] or response in accepted_answers[1]:
                    return response
                else:
                    print(attempts, response, accepted_answers[0], accepted_answers[1])
                    attempts += 1
                    time.sleep(5)
            else:
                attempts += 1
                print('Status code {}'.format(response.status_code))
                time.sleep(60 if str(response.status_code) == '429' else 5)  # Wait for 5 seconds before retrying
        except requests.exceptions.Timeout as e:
            attempts += 1  # Retry if the request times out
            time.sleep(5)
            print(attempts, e)
        except requests.exceptions.RequestException as e:
            attempts += 1
            time.sleep(5)  # Wait for 5 seconds before retrying
            print(attempts, e)
    print(prompt)
    raise Exception('max_attempts exceeded')
    return None  # Return None if all attempts fail

# Function to perform pairwise comparison for disciplines
def compare_disciplines(abstract, disciplines, base_folder):
    discipline_scores = {discipline: 0 for discipline in disciplines}
    for discipline1, discipline2 in itertools.combinations(disciplines, 2):
        prompt_file = f"{discipline1}_vs_{discipline2}.txt"
        prompt_path = os.path.join(base_folder, prompt_file)
        if os.path.exists(prompt_path):
            with open(prompt_path, 'r') as file:
                prompt = file.read().format(abstract)
                selected_discipline = gpt_pairwise_comparison_with_retry(prompt, accepted_answers=(discipline1, discipline2))
                if selected_discipline in discipline1:
                    discipline_scores[discipline1] += 1
                elif selected_discipline in discipline2:
                    discipline_scores[discipline2] += 1
    return discipline_scores

# Function to perform pairwise comparison for subdisciplines
def compare_subdisciplines(abstract, discipline, subdisciplines, base_folder):
    subdiscipline_scores = {subdiscipline: 0 for subdiscipline in subdisciplines}
    discipline_folder = os.path.join(base_folder, discipline)
    for sub1, sub2 in itertools.combinations(subdisciplines, 2):
        prompt_file = f"{sub1}_vs_{sub2}.txt"
        prompt_path = os.path.join(discipline_folder, prompt_file)
        if os.path.exists(prompt_path):
            with open(prompt_path, 'r') as file:
                prompt = file.read().format(abstract)
                selected_subdiscipline = gpt_pairwise_comparison_with_retry(prompt, accepted_answers=(sub1, sub2))
                if selected_subdiscipline in sub1:
                    subdiscipline_scores[sub1] += 1
                elif selected_subdiscipline in sub2:
                    subdiscipline_scores[sub2] += 1
    return subdiscipline_scores

# Define the main processing function
def process_publications(test_file_path, base_prompt_folder, discipline_data, output_file_path, api_key):
    # Try to load existing results from the JSON file
    if os.path.exists(output_file_path):
        with open(output_file_path, 'r') as json_file:
            results_dict = json.load(json_file)
    else:
        results_dict = {}
    
    count = 0
    # Read the test file and process each publication
    with open(test_file_path, mode='r') as file:
        csv_reader = csv.DictReader(file)
        for row in tqdm(csv_reader, total=191):
            try:
                pubid = row['PUBID']

                # Skip processing if results already exist for this publication
                if pubid in results_dict:
                    continue
                count += 1
                abstract = row['ABSTRACT']
                disciplines = list(discipline_data.keys())
                discipline_scores = compare_disciplines(abstract, disciplines, base_prompt_folder)

                # Compare subdisciplines for selected disciplines
                for discipline, score in discipline_scores.items():
                    if score >= 4 and discipline in discipline_data:
                        subdisciplines = list(discipline_data[discipline].keys())
                        subdiscipline_scores = compare_subdisciplines(abstract, discipline, subdisciplines, base_prompt_folder)
                        discipline_scores[discipline] = {'score': score, 'subdisciplines': subdiscipline_scores}
                    else:
                        discipline_scores[discipline] = {'score': score, 'subdisciplines': {}}

                # Update results
                results_dict[pubid] = discipline_scores
                with open(output_file_path, 'w') as json_file:
                    json.dump(results_dict, json_file, indent=4)
            except Exception as e:
                print(e)
                continue
        print(count)

# Paths and API key
test_file_path = 'H:/data/WoS_data/data_gpt10.csv'  # Replace with the actual path
base_prompt_folder = 'prompts/pairwise_comparison'  # Replace with the actual path
output_file_path = 'chatgpt_hierarchical_pairwise.json'  # Replace with the actual path
api_key = 'api-key'  # Replace with your actual API key
discipline_data = json.load(open('discipline_structure.json', 'r'))

# Process the publications
process_publications(test_file_path, base_prompt_folder, discipline_data, output_file_path, api_key)

  0%|                                                                                          | 0/191 [00:00<?, ?it/s]

0 the most relevant discipline is 'electrical engineering, electronic engineering, information engineering electrical engineering, electronic engineering, information engineering mechanical engineering


  3%|██                                                                              | 5/191 [02:25<1:37:47, 31.54s/it]

1 HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=60)


  3%|██▌                                                                             | 6/191 [04:15<2:51:53, 55.75s/it]

0 industrial biotechnology mechanical engineering medical engineering
0 this abstract is more relevant to 'biological sciences earth and related environmental sciences biological sciences


  4%|██▉                                                                             | 7/191 [05:42<2:30:12, 48.98s/it]


KeyboardInterrupt: 

In [6]:
import json
with open('chatgpt_hierarchical_pairwise.json', 'r') as json_file:
    results_dict = json.load(json_file)

In [10]:
len(results_dict.keys())

20

In [7]:
response = 'engineering and technology'
accepted_answers = {'engineering and technology', 'medical and health sciences'}

In [8]:
any(response in x for x in accepted_answers)

True

In [10]:
response

'engineering and technology'

In [11]:
accepted_answers

{'engineering and technology', 'medical and health sciences'}

In [23]:
a

"'engineering and technology'"

In [24]:
b

'engineering and technology'

In [25]:
c

'natural sciences'